# <Fast text를 이용한 유사도 계산>

In [2]:
import gzip

## - 필요한 모듈 import

In [3]:
from gensim import models
ko_model = models.fasttext.load_facebook_model('cc.ko.300.bin.gz')
from sklearn.decomposition import PCA
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
from konlpy.tag import Okt
import nltk

## - 새로운 글 & 기존 글(상위 5개) 토큰 사이 유사도 검사 
### 1. 데이터 불러오기

In [152]:
data = pd.read_excel('finished_petition.xlsx')[['제목', '청원수', '본문']]
okt = Okt()

delete_word = ['현재', '지금', '상반기', '하반기', '이', '그', '그녀', '저', '것', '들', '제', '저희', '너', '되', '물', '수', '않', '없', '아니',
               '때문', '곳', '등', '들', '중', '좀', '잘', '더', '더욱', '을해', '자주', '경우', '전혀', '올해', '말', '다시', '대해', '후', '때', '있', '하', '생각', '청원', '국민청원']

final_token = []
new_final_token = []
df = pd.DataFrame(columns=['청원글', '청원수', '상위토큰', 'fasttext 유사도'])

### 2. 인풋 받기

In [153]:
# 인풋
new_petition = okt.nouns(input("새로 청원할 글을 입력하세요: "))

새로 청원할 글을 입력하세요: 미세먼지가 10년전에 비해 상당히 자주 몰려오고있습니다. 언론에서도 중국발이라고 얘기는 하고있다만 가장중요한 정부기관이 중국에대해 일절 말한마디조차 하지않고 되려 중국과 상호협력하여 미세먼지를 줄이자는 대책뿐입니다. 왜 상호협력이죠? 지금 저희 한국은 엄연히 중국에서 뿜어져나오는 미세먼지로인하여 계속해서 국민들이 죽어나가고있습니다.... 비유를하자면 범죄자랑 같이 범죄예방하는거랑 다름이없는거죠 지금 국제보건기구에 이 진상을 알려 국민들도 모든 전세계들이 중국의 더러운짓을 만천하에 고발을해야할것입니다. 환경단체들이 아무리 집회를열고 국민청원을 아무리열심히넣어도 변하지않습니다... 왜냐하면 지금 제일 일을해야할 환경부조차 아무소리안하고 대통령님또한 미세먼지에대해 전혀언급이없기때문에 국민들은 불신하고있습니다... 제발 중국에 대해 항의를하시고 더불어 산둥반도에 위치한 공장들을 폐쇄하라고 말해주십시요 이것은 국민의 건강과 수명 및 미래가 달린일입니다. 미세먼지는 절대 좋은것도아니며 중금속 입자이기때문에 방사능과 같은 효과를 가진 매우위험한 물질입니다. 마스크만쓴다고 되는게 전혀아닙니다!!!!!! 중국에서 헛소리지껄이면서 할수없다고하면 단호히 단교하고 국제소송을 걸어야할것입니다


### 3. 새로운 청원글 토큰화

In [154]:
# 새로운 청원글 토큰화
new_text = nltk.Text(new_petition)
new_selected_words = [f[0] for f in new_text.vocab().most_common(15)]
print(new_selected_words)

new_final_token = []
# 새로운 청원글 불용어 제거
for word in new_selected_words:
    if word not in delete_word:
        new_final_token.append(word)
print(new_final_token)

new_final_token = new_final_token[:7]


['중국', '미세먼지', '국민', '대해', '지금', '말', '상호', '협력', '국제', '을해', '전혀', '때문', '것', '자주', '언론']
['중국', '미세먼지', '국민', '상호', '협력', '국제', '언론']


In [155]:
new_final_token

['중국', '미세먼지', '국민', '상호', '협력', '국제', '언론']

### 4. 기존 청원글 토큰화
### 5. 제목이랑 본문상위토큰 합치기
### 6. 불용어 제거하기 -> 5개 최종 토큰만 추출하여 final_token에 저장
### 7. 새로운 청원글 & 기존 청원글 명사토큰 1:1 유사도 검사

In [156]:
similarity_rate = 0
similarity_rate_hap = 0
max_similarity_rate = 0

# 3. 기존 청원글 토큰화

# for i in range(10):
for i in range(data['제목'].count()):
    # 제목 토큰
    title_token = okt.nouns(data['제목'][i])
    # 콘텐트 토큰
    content_token = okt.nouns(data['본문'][i])
    text = nltk.Text(content_token)
    # 출현 빈도가 높은 본문 상위 토큰 15개
    selected_words = [f[0] for f in text.vocab().most_common(15)]
    
# 4. 제목이랑 본문상위토큰 15개 합치기
#     hap_token = title_token + selected_words
    hap_token = selected_words
#     print("제목 토큰 & 본문에서 가장 많이 나온 명사 15개: \n", hap_token)
    
# 5. 불용어 제거하기
    for word in hap_token:
        if word not in delete_word:
            final_token.append(word)
            
# 5-2. 불용어 제거한 후, 7개 토큰만 추출
    final_token = final_token[:7]
#     print("불용어 제거한 최종 토큰: \n", final_token)

    
# 6. 새로운 글 & 기존의 청원글과 fasttext 유사도 검사
    def fasttext_similarity(final_token, new_final_token):
        global rates
        global max_number
        
        for t in range(7):
            rates = []
            max_number = 0
            
            for k in range(7):       
                similarity_rate = ko_model.wv.similarity(new_final_token[t], final_token[k]) # 1:1 유사도
                rates.append(similarity_rate)
                
            for a in range(len(rates)):
                if max_number <= rates[a]:
                    max_number = rates[a]
            max_similarity_rate = max_number
            print('새로운 글의 ', t+1 , '번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도:', max_similarity_rate)
            
            global similarity_rate_hap
            similarity_rate_hap += max_similarity_rate
            
        print('유사도 합:', similarity_rate_hap)

        print('------------------------------------')
        return similarity_rate_hap

    df = df.append({'청원글':data['제목'][i], '청원수': data['청원수'][i], 
                '상위토큰': final_token, 'fasttext 유사도': fasttext_similarity(final_token, new_final_token)}, 
               ignore_index=True)

    final_token = []
    similarity_rate = 0
    similarity_rate_hap = 0

새로운 글의  1 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.23395295
새로운 글의  2 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.20704661
새로운 글의  3 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.5101544
새로운 글의  4 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.22009782
새로운 글의  5 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.2602318
새로운 글의  6 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.21879691
새로운 글의  7 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.27552542
유사도 합: 1.9258059412240982
------------------------------------
새로운 글의  1 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.21891159
새로운 글의  2 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.17200184
새로운 글의  3 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.30889884
새로운 글의  4 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.19989924
새로운 글의  5 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.2316229
새로운 글의  6 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.23079276
새로운 글의  7 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.27406904
유사도 합: 1.6361962109804153
------------------------------------
새로운 글의  1 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.2886804
새로운 글의  2 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.24018133
새로운 글의  3 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.2592377
새로운 글의  4 번째

새로운 글의  1 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.3164072
새로운 글의  2 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.17408873
새로운 글의  3 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.41785422
새로운 글의  4 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.19317257
새로운 글의  5 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.27384397
새로운 글의  6 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.32951164
새로운 글의  7 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.32252356
유사도 합: 2.0274019092321396
------------------------------------
새로운 글의  1 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.37575287
새로운 글의  2 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.23956323
새로운 글의  3 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.3000866
새로운 글의  4 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.2083028
새로운 글의  5 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.30012596
새로운 글의  6 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.28057295
새로운 글의  7 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.29930556
유사도 합: 2.0037099421024323
------------------------------------
새로운 글의  1 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 1.0
새로운 글의  2 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.32461008
새로운 글의  3 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 1.0
새로운 글의  4 번째 토큰과 기존의 글의 

새로운 글의  1 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.30890304
새로운 글의  2 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.09078652
새로운 글의  3 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.32879716
새로운 글의  4 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.29363948
새로운 글의  5 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.34924173
새로운 글의  6 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.24154383
새로운 글의  7 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.44648322
유사도 합: 2.059394985437393
------------------------------------
새로운 글의  1 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.24525031
새로운 글의  2 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.12049892
새로운 글의  3 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.24724841
새로운 글의  4 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.16103709
새로운 글의  5 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.32590786
새로운 글의  6 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.2581004
새로운 글의  7 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.30540726
유사도 합: 1.6634502336382866
------------------------------------
새로운 글의  1 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.25131398
새로운 글의  2 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.17613122
새로운 글의  3 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.35310465
새로운 글의  4

새로운 글의  1 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.3164072
새로운 글의  2 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.17408873
새로운 글의  3 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 1.0
새로운 글의  4 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.21519445
새로운 글의  5 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.295622
새로운 글의  6 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.24960157
새로운 글의  7 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.28991002
유사도 합: 2.5408239662647247
------------------------------------
새로운 글의  1 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.26082852
새로운 글의  2 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.15223116
새로운 글의  3 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.22343314
새로운 글의  4 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.2143734
새로운 글의  5 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.28042427
새로운 글의  6 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.32613048
새로운 글의  7 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.5655518
유사도 합: 2.0229727774858475
------------------------------------
새로운 글의  1 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.29474
새로운 글의  2 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.16419852
새로운 글의  3 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.3376446
새로운 글의  4 번째 토큰과 기존의 글의

새로운 글의  1 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.40342504
새로운 글의  2 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.24221939
새로운 글의  3 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.508859
새로운 글의  4 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.23118462
새로운 글의  5 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.36858064
새로운 글의  6 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.40962997
새로운 글의  7 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.39027932
유사도 합: 2.5541779547929764
------------------------------------
새로운 글의  1 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.28922057
새로운 글의  2 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.17451663
새로운 글의  3 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.40692776
새로운 글의  4 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.22626041
새로운 글의  5 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.29546994
새로운 글의  6 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.4077456
새로운 글의  7 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.35940868
유사도 합: 2.159549593925476
------------------------------------
새로운 글의  1 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.40342504
새로운 글의  2 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.17017597
새로운 글의  3 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.39828482
새로운 글의  4 번

새로운 글의  1 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.24525031
새로운 글의  2 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.17017597
새로운 글의  3 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.21804376
새로운 글의  4 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.18710884
새로운 글의  5 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.2816289
새로운 글의  6 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.25013682
새로운 글의  7 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.24760798
유사도 합: 1.599952593445778
------------------------------------
새로운 글의  1 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.27480626
새로운 글의  2 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.15178299
새로운 글의  3 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 1.0
새로운 글의  4 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.19317257
새로운 글의  5 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.27384397
새로운 글의  6 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.23984471
새로운 글의  7 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.28991002
유사도 합: 2.423360526561737
------------------------------------
새로운 글의  1 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.21214493
새로운 글의  2 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.24378827
새로운 글의  3 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.25413507
새로운 글의  4 번째 토큰과 

새로운 글의  4 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.2004181
새로운 글의  5 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.32790864
새로운 글의  6 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.4065064
새로운 글의  7 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.42579928
유사도 합: 2.2411026656627655
------------------------------------
새로운 글의  1 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.30932465
새로운 글의  2 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.21647549
새로운 글의  3 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.40692776
새로운 글의  4 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.2004181
새로운 글의  5 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.32790864
새로운 글의  6 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.4077456
새로운 글의  7 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.42579928
유사도 합: 2.2945995181798935
------------------------------------
새로운 글의  1 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.17982826
새로운 글의  2 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.20704661
새로운 글의  3 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.21804376
새로운 글의  4 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.1448909
새로운 글의  5 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.16036676
새로운 글의  6 번째 토큰과 기존의 글의 토큰 간 가장 높은 유사도: 0.20315793
새로운 글의  7 번째

In [157]:
df

,청원글,청원수,상위토큰,fasttext 유사도
0,저희 아파트 경비아저씨의 억울함을 풀어주세요...,"446,434","[아저씨, 아파트, 주민, 분, 입, 정말, 경비]",1.925806
1,"울산 초등학교 1학년 아이에게 팬티빨기 숙제내고 학생 사진에 '섹시팬티', '공주님...","225,764","[아이, 교사, 팬티, 씨, 사진, 성적, 숙제]",1.636196
2,등교 개학 시기를 미루어주시기 바랍니다.,"255,333","[개학, 등교, 확, 진자, 코로나, 학교, 감염]",1.882635
3,방송통신위원회는 방송의 공적 책임 방기하고 언론이기를 포기한 채널A와 TV조선의 ...,"273,513","[채널, 승인, 방송, 재, 방송통신, 위원회, 취재]",2.046758
4,렌트카 훔쳐 사망사고를 낸 10대 엄중 처벌해주세요,"1,007,040","[청소년, 경찰, 사망, 처벌, 형사, 오전, 대전]",1.873434
...,...,...,...,...
128,나경원 의원 평창올림픽 위원직을 파면시켜주세요,"360,905","[올림픽, 평창, 국민, 평창올림픽, 나경원, 의원, 대한민국]",2.577399
129,"김보름, 박지우 선수의 자격박탈과 적폐 빙상연맹의 엄중 처벌을 청원합니다","614,127","[선수, 연맹, 오늘, 김보름, 박지우, 국가, 올림픽]",2.360577
130,미성년자 성폭행 형량 올려주세요 종신형원합니다,"233,842","[사건, 또, 술, 형량, 애, 창원, 조두순]",1.522531
131,초.중.고 학교 페미니즘교육 의무화,"213,219","[학생, 학교, 단어, 선생님, 아이, 알, 페미니즘]",1.907889


In [158]:
sorted_df = df.sort_values(by = ['fasttext 유사도'], axis = 0, ascending=False, inplace=False)  #fasttext 유사도 기준 내림차순으로 정렬
sorted_df

,청원글,청원수,상위토큰,fasttext 유사도
114,미세먼지의 위험 그리고 오염 및 중국에대한 항의,"278,128","[중국, 미세먼지, 국민, 상호, 협력, 국제, 언론]",7.000000
52,전 언론의 세무조사를 명령한다.,"227,314","[언론사, 국민, 언론, 대한민국, 국가, 동의, 국세청]",3.521823
23,문재인 대통령 탄핵을 촉구합니다.,"1,469,023","[입국, 대통령, 금지, 중국, 마스크, 국민, 조치]",3.472123
20,한전 사업에 중국 기업의 참여를 허락하는 것은 말도 안됩니다.,"383,039","[기업, 중국, 국내, 공기업, 입찰, 기회, 사업]",3.123727
76,자유 한국당 정당해산 청원,"1,831,900","[정부, 자유, 한국, 국민, 의원, 국회의원, 정당]",3.112163
...,...,...,...,...
66,아동 성폭행범을 감형한 *** 판사 파면하라,"240,298","[판사, 가해자, 상대로, 처벌, 이유, 성범죄, 아이]",1.476358
32,N번방 대화 참여자들도 명단을 공개하고 처벌해주십시오,"457,487","[처벌, 참여자, 아이, 대화방, 운영자, 미성년, 감]",1.422539
15,초등학생을 성폭행하고 찍은 불법촬영물로 협박해 금품을 갈취하여 인생을 망가뜨린 고등...,"354,260","[가해자, 피해자, 아이, 처벌, 이유, 나이, 성폭행]",1.386880
125,대전 아파트 단지내 횡단보도 교통사고..가해자의 만행과 도로교통법의 허점,"219,395","[가해자, 아이, 횡단보도, 사고, 딸아이, 아파트, 단지]",1.353335
